In [ ]:

"""
Generate DFT images
"""

import cv2
import os
import torch
import random
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import foolbox as fb
from keras.datasets import cifar100

In [ ]:
def generate_dft(model, data, label, image_type, output_name):
    """
    model: target model to adversarial attack
    data: seem images
    label: label of model
    image_type: type for adversarial DFT
    """
    if image_type == 'test':
        scope = range(9)
    elif image_type == 'val':
        scope = range(9,18)
    elif image_type == 'train':
        scope = range(18,68)
    
    filepath_temp = './images/temp'
    filepath ='./images/'+image_type+'/'+label
    if not os.path.exists(filepath_temp):
        os.makedirs(filepath_temp)
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    for k in scope:
        X_test=torch.from_numpy(data[0+32*k:32+32*k]).float().to(device)
        X_test=F.interpolate(X_test, size=(224, 224), mode='bicubic', align_corners=False)
        y_test=[]
        for j in range(32):
            y_test.append(torch.argmax(model(X_test)[j]))
        y_test=torch.tensor(y_test).to(device)
        attack = fb.attacks.FGSM()
        bounds = (-1, 50)
        fmodel = fb.PyTorchModel(model, bounds=bounds)
        fmodel = fmodel.transform_bounds((-1, 50))
        raw, clipped, is_adv = attack(fmodel,X_test,y_test, epsilons=0.03)

        for i in range(32):
            plt.figure(num=None, figsize=(4,3), dpi=150)
            plt.figure(figsize = (2,2))
            plt.imshow((clipped[i]-X_test[i]).cpu().permute(1,2,0));
            plt.axis('off')
            plt.savefig(filepath_temp+'/per'+str(i+32*k)+'.jpg', dpi=150,bbox_inches='tight', pad_inches=0)
        for i in range(32):
            img_c1=cv2.imread(os.path.join(filepath_temp,'per'+str(i+32*k)+'.jpg'), 0)
            img_c2 = np.fft.fft2(img_c1)
            img_c3 = np.fft.fftshift(img_c2)
            cv2.imwrite(os.path.join(filepath,output_name+str(i+32*k)+'.jpg'),20*np.log(1+np.abs(img_c3)))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#load dataset
(X_train, y_train), (X_test1, y_test1) = cifar100.load_data()
X_test1=X_test1.reshape(10000,3,32,32)
X_test1=X_test1/255
X_test1=X_test1.astype(np.float32)
random.seed(10)
random_numbers = np.arange(10000)
random.shuffle(random_numbers)
X_test1 = X_test1[random_numbers]

In [ ]:
#Train architectures and datasets for DeepTaster
architectures = ['Resnet18','Vgg16','Densenet161']
datasets = ['Cifar10','MNIST','Tiny']

In [ ]:
#Test dataset for DeepTaster
for dataset in range(3):
    for architecture in architectures:
        model=torch.load("./models/test/"+datasets[dataset]+"_"+architecture+".pt",map_location='cpu')
        model = model.to(device)
        model.eval()
        generate_dft(model, X_test1,  str(dataset), 'test', architecture)


In [ ]:
#Train dataset for DeepTaster    
for architecture in architectures :
    #Load Pretrained model
    model=torch.load("./models/victims/Cifar10_"+architecture+".pt",map_location='cpu')
    model = model.to(device)
    model.eval()
    generate_dft(model, X_test1, '0', 'train', architecture)


In [ ]:
#validattion dataset for DeepTaster
for architecture in architectures:
    model=torch.load("./models/victims/Cifar10_"+architecture+".pt",map_location='cpu')
    model = model.to(device)
    model.eval()
    generate_dft(model, X_test1, "0", 'val', architecture)